In [12]:
from subprocess import check_output
print(check_output(["ls", "./data"]).decode("utf8"))

members.csv
sample_submission.csv
song_extra_info.csv
songs.csv
test.csv
train.csv



In [23]:
import numpy as np
import collections

In [14]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load data

In [52]:
date_columns = ['expiration_date', 'registration_init_time']

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
songs = pd.read_csv('data/songs.csv')
members = pd.read_csv('data/members.csv', parse_dates=date_columns)
song_extra_info = pd.read_csv('data/song_extra_info.csv')

# Merge data

In [16]:
all_data = pd.concat([train, test], sort=True)

all_data = all_data.merge(songs, on='song_id', how='left')
all_data = all_data.merge(members, on='msno', how='left')

# Replace NA

In [45]:
enc = LabelEncoder()

for col in [
    'msno', 'song_id', 'source_screen_name',
    'source_system_tab', 'source_type', 'genre_ids',
    'artist_name', 'composer', 'lyricist', 'gender'
]:
    all_data[col] = enc.fit_transform(all_data[col].fillna('nan'))

In [54]:
for col in ['language', 'city', 'registered_via']:
    all_data[col] = enc.fit_transform(all_data[col].fillna(-2))

# Split Data

In [59]:
n = len(train)
train_data = all_data[:n]
test_data = all_data[n:]

# Replace age outlier

In [73]:
members_filtered = members[(members['bd'] > 0) & (
    members['bd'] < 100)]    # drop age<0 and age>100

bd_mean = np.mean(members_filtered['bd'])

In [74]:
train_data.loc[(train_data['bd'] <= 0) | (
    train_data['bd'] > 100), 'bd'] = bd_mean

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [70]:
train_data.bd.describe()

count    7.377418e+06
mean     2.879205e+01
std      6.691515e+00
min      2.000000e+00
25%      2.600000e+01
50%      2.889735e+01
75%      2.900000e+01
max      9.500000e+01
Name: bd, dtype: float64

In [71]:
train_data

,id,msno,song_id,source_screen_name,source_system_tab,source_type,target,song_length,genre_ids,artist_name,composer,lyricist,language,city,bd,gender,registered_via,registration_init_time,expiration_date
0,NaN,9176.0,86884.0,7.0,1.0,7.0,1.0,206471.0,307.0,3784.0,16653.0,26024.0,9.0,0.0,28.897349,2.0,2.0,2012-01-02 00:00:00,2017-10-05 00:00:00
1,NaN,19273.0,260594.0,8.0,3.0,4.0,1.0,284584.0,97.0,36867.0,74276.0,26024.0,9.0,11.0,24.000000,0.0,3.0,2011-05-25 00:00:00,2017-09-11 00:00:00
2,NaN,19273.0,140755.0,8.0,3.0,4.0,1.0,225396.0,97.0,24601.0,51540.0,26024.0,9.0,11.0,24.000000,0.0,3.0,2011-05-25 00:00:00,2017-09-11 00:00:00
3,NaN,19273.0,27577.0,8.0,3.0,4.0,1.0,255512.0,6.0,31651.0,41991.0,26024.0,1.0,11.0,24.000000,0.0,3.0,2011-05-25 00:00:00,2017-09-11 00:00:00
4,NaN,9176.0,38706.0,7.0,1.0,7.0,1.0,187802.0,2.0,5190.0,9701.0,26024.0,9.0,0.0,28.897349,2.0,2.0,2012-01-02 00:00:00,2017-10-05 00:00:00
5,NaN,9176.0,35087.0,7.0,1.0,7.0,1.0,247803.0,97.0,9452.0,63673.0,26024.0,9.0,0.0,28.897349,2.0,2.0,2012-01-02 00:00:00,2017-10-05 00:00:00
6,NaN,19273.0,221459.0,8.0,3.0,4.0,1.0,229982.0,371.0,3348.0,66331.0,22941.0,6.0,11.0,24.000000,0.0,3.0,2011-05-25 00:00:00,2017-09-11 00:00:00
7,NaN,9176.0,258674.0,7.0,1.0,7.0,1.0,181115.0,2.0,35342.0,67760.0,26024.0,9.0,0.0,28.897349,2.0,2.0,2012-01-02 00:00:00,2017-10-05 00:00:00
8,NaN,31394.0,10642.0,8.0,3.0,3.0,1.0,278964.0,200.0,25767.0,60384.0,26024.0,9.0,13.0,26.000000,1.0,3.0,2011-11-07 00:00:00,2018-03-04 00:00:00
9,NaN,31394.0,109197.0,8.0,3.0,3.0,1.0,257369.0,371.0,25767.0,60384.0,26024.0,9.0,13.0,26.000000,1.0,3.0,2011-11-07 00:00:00,2018-03-04 00:00:00
